In [1]:
import pickle
import numpy as np
import pandas as pd

DATA = {
    "X" : [],
    "category" : [],
    "y" : []
}

X=DATA["X"]
category=DATA["category"]
y=DATA["y"]


with open('covid_fake_news_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(0,len(dst)))
    data = pd.read_csv("raw/covid_fake_news.csv",sep="\t")
    y.extend(data["label"].values)
    
with open('mmcovid_en_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(1,len(dst)))
    data = pd.read_csv("raw/mmcovid_en.csv",sep=",")
    y.extend(data["label"].values)
    

with open('pubhealth_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(2,len(dst)))
    data = pd.read_csv("raw/pubhealth.csv",sep=",")
    y.extend(data["label"].values)

    
with open('qprop_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(3,len(dst)))
    data = pd.read_csv("raw/qprop.csv",sep="\t")
    y.extend(data["label"].values)
    
with open('isot_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(4,len(dst)))
    data = pd.read_csv("raw/isot.csv",sep=",")
    y.extend(data["label"].values)
    
with open('grafn_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(5,len(dst)))
    data = pd.read_csv("raw/grafn.csv",sep=",")
    y.extend(data["label"].values)
    
    
DATA["X"]=np.array(X)
DATA["category"]=np.array(category)
DATA["y"]=np.array(y)
DATA["folds"] = []

/tmp/ipykernel_2845/3486400938.py:57: DtypeWarning: Columns (2,4,5,8,9,10,11,12,13,15,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("raw/grafn.csv",sep=",")


In [2]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.metrics import geometric_mean_score
from IPython.display import clear_output

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def experiment(foldids, X, y, cls = LogisticRegression(max_iter=10000), fit=True):

    scores = {
        'Accuracy': {'func': accuracy_score},
        'Balanced Accuracy': {'func': balanced_accuracy_score},
        'F1': {'func': f1_score},
        'Precision': {'func': precision_score},
        'Recall': {'func': recall_score},
        'G-mean': {'func': geometric_mean_score}
    }

    for score_name, score_dict in scores.items():
        scores[score_name]["list"] = []
        scores[score_name]["lab"] = []

    for fold,j in enumerate(foldids):
        train = foldids[fold][1]
        test = foldids[fold][2]
        xin, yin = X[train], np.array(y[train])
        
        pca = PCA(n_components=128)
        pca.fit(xin)
        
        
        if fit == True:
            cls.fit(pca.transform(xin), yin)
        y_pred = cls.predict(pca.transform(X[test]))
        for score_name, score_dict in scores.items():
            if score_name in ["F1","Precision","Recall"]:
                scorvaln = score_dict['func'](y[test], y_pred, average=None)
                score_dict['lab'].append(scorvaln)
                scorval = score_dict['func'](y[test], y_pred, average="weighted")
                score_dict['list'].append(scorval)
                #print(score_name, scorval, scorvaln)  
            else:
                scorval=score_dict['func'](y[test], y_pred)
                score_dict['list'].append(scorval)
                #print(score_name, scorval)
        #print(" ")

    #clear_output()
    for score_name, score_dict in scores.items():
        score_dict['avg'] = np.mean(score_dict['list'])
        score_dict['std'] = np.std(score_dict['list'])
 
    # Print stats
    numlabels = scores["F1"]["lab"][0].shape[0]
    scores["F1"]["lab"][0].shape[0] 
    head = "| %-20s | %-10s |" +  numlabels * " %-10s |" 
    headv = ["Score", "Average"]
    headv.extend(["Kat_"+str(i+1) for i in range(numlabels)])
    row=head % tuple(headv)
    print("+"*len(row))
    print(row)
    print("+"*len(row))
    for score_name, score_dict in sorted(scores.items()) :
        headv = [score_name, np.mean(score_dict['list'])*100, np.std(score_dict['list'])*100]
        for i in range(numlabels):
            if score_name in ["F1","Precision","Recall"]:
                head = "| %-20s | %4.1f ± %3.1f |" + numlabels* " %4.1f ± %3.1f |"
                vals = [v[i] for v in scores[score_name]["lab"]]
                headv.append(np.mean(vals)*100)
                headv.append(np.std(vals)*100)
            else:
                head = "| %-20s | %4.1f ± %3.1f |" + numlabels * " %-10s |" 
                headv.append("-")
        print(head % tuple(headv))
    print("+"*len(row))
    return cls, scores, pca

<hr style="border: 5px dashed red">

# Single Task Performance

In [22]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.decomposition import PCA

DATA["folds"] = []
rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)

stl=[]

for t in range(np.max(DATA["category"]+1)):
    print("TASK",t)
    X = DATA["X"][DATA["category"] == t]
    y = DATA["y"][DATA["category"] == t]

    foldids = []
    for fold_idx, (train, test) in enumerate(rskf.split(X, y)):
        foldids.append((fold_idx,train,test))

    #print("shapes X",X.shape,"y", y.shape)
    DATA["folds"].append(foldids)
    model1, scores1, pca = experiment(foldids, X, y, 
                                 LogisticRegression(max_iter=10000))
    stl.append((model1,pca,scores1))

TASK 0
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Score                | Average    | Kat_1      | Kat_2      |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Accuracy             | 96.9 ± 0.1 | -          | -          |
| Balanced Accuracy    | 77.2 ± 1.3 | -          | -          |
| F1                   | 96.7 ± 0.2 | 98.4 ± 0.1 | 65.0 ± 1.8 |
| G-mean               | 74.0 ± 1.7 | -          | -          |
| Precision            | 96.7 ± 0.2 | 97.6 ± 0.1 | 79.2 ± 3.0 |
| Recall               | 96.9 ± 0.1 | 99.2 ± 0.2 | 55.2 ± 2.6 |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
TASK 1
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Score                | Average    | Kat_1      | Kat_2      |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Accuracy             | 92.2 ± 0.3 | -          | -          |
| Balanced Accuracy    | 87.6 ± 0.8 | -          | -          |
| F1                   | 9

In [28]:
T = np.max(DATA["category"])+1
d = 128

stl_bacs = [[] for b in range(T)]

ft = 0

for ft in range(6):
    print("STL-",ft)
    for _foldid in range(10):
        #print("fold",_foldid)

        ##### Test
        for task in range(T):
            X = DATA["X"][DATA["category"] == task]
            y = DATA["y"][DATA["category"] == task]
            foldids = DATA["folds"][task]

            train = foldids[_foldid][1]
            test = foldids[_foldid][2]    

            X=stl[ft][1].transform(X[test])

            #print(np.unique(domain_pred, return_counts=True))
            #print("task",task, pred_task_id)

            pred=stl[ft][0].predict(X)
            bac = balanced_accuracy_score(y[test], pred)
            #print(task,bac)
            stl_bacs[task].append(bac)
            


STL- 0
STL- 1
STL- 2
STL- 3
STL- 4
STL- 5
50.1 ± 0.43 &
68.9 ± 0.42 &
41.3 ± 0.52 &
77.3 ± 0.21 &
89.4 ± 0.15 &
74.1 ± 0.11 &
---
66.86956697284273 16.370773622340355


In [29]:
for t in range(T):
    print("%3.1f ± %3.2f &" % (np.mean(stl_bacs[t])*100, 100*np.std(stl_bacs[t])) )

print("---")
print(np.mean(np.array(stl_bacs).flatten())*100, np.std(np.array(stl_bacs).flatten())*100)

50.8 ± 13.78 &
56.9 ± 15.20 &
48.1 ± 14.90 &
57.0 ± 17.17 &
58.7 ± 23.76 &
54.7 ± 14.58 &
---
54.36844556864055 17.319565221417843


# Batch Multi Task Performance

In [4]:
X = DATA["X"]
y = DATA["y"]

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

model3, scores3, model3_pca = experiment(foldids, X, y, 
                             LogisticRegression(max_iter=10000))

100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 169.94it/s]

shapes X (186477, 768) y (186477,)


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Score                | Average    | Kat_1      | Kat_2      |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Accuracy             | 84.5 ± 0.1 | -          | -          |
| Balanced Accuracy    | 77.7 ± 0.1 | -          | -          |
| F1                   | 84.0 ± 0.1 | 89.7 ± 0.1 | 68.9 ± 0.2 |
| G-mean               | 76.3 ± 0.2 | -          | -          |
| Precision            | 84.0 ± 0.1 | 86.9 ± 0.1 | 76.3 ± 0.3 |
| Recall               | 84.5 ± 0.1 | 92.7 ± 0.1 | 62.8 ± 0.3 |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [5]:
T = np.max(DATA["category"])+1
d = 128

_bacs = [[] for b in range(T)]


for _foldid in range(10):
    print("fold",_foldid)

    ##### Test
    for task in range(T):
        X = DATA["X"][DATA["category"] == task]
        y = DATA["y"][DATA["category"] == task]
        foldids = DATA["folds"][task]
        
        train = foldids[_foldid][1]
        test = foldids[_foldid][2]    
        
        X=model3_pca.transform(X[test])

        #print(np.unique(domain_pred, return_counts=True))
        #print("task",task, pred_task_id)

        pred=model3.predict(X)
        bac = balanced_accuracy_score(y[test], pred)
        print(task,bac)
        _bacs[task].append(bac)



fold 0
0 0.49759623242889833
1 0.6827648219050935
2 0.42324744822202087
3 0.7740426345626505
4 0.8954930213517331
5 0.7415292180821338
fold 1
0 0.5046861090339351
1 0.6956143404107205
2 0.40334829229467034
3 0.7721165139737413
4 0.8929018452055881
5 0.7409854492895602
fold 2
0 0.507643429225237
1 0.6899727346559925
2 0.4081338054532713
3 0.7723664995930211
4 0.893690589254943
5 0.7411778912232476
fold 3
0 0.4946861599035512
1 0.6884064276598213
2 0.4184691154516677
3 0.7737926489433709
4 0.8947044891947944
5 0.7413364206470661
fold 4
0 0.495481570774763
1 0.6918581041884209
2 0.41274118119158965
3 0.7720576816661714
4 0.8970492567177326
5 0.741581899741727
fold 5
0 0.5068012676708329
1 0.6865210581273931
2 0.41386047258601155
3 0.7741014668702204
4 0.8913456106829852
5 0.7409325319905253
fold 6
0 0.499711915658712
1 0.682822833275322
2 0.41168017323402994
3 0.7746342426910564
4 0.8953652639205092
5 0.7428369103898261
fold 7
0 0.5025897721549896
1 0.6955563290404919
2 0.4149220437112769

In [19]:
for t in range(T):
    print("%3.1f ± %3.2f &" % (np.mean(_bacs[t])*100, 100*np.std(_bacs[t])) )

print("---")
print(np.mean(np.array(_bacs).flatten())*100, np.std(np.array(_bacs).flatten())*100)

50.1 ± 0.43 &
68.9 ± 0.42 &
41.3 ± 0.52 &
77.3 ± 0.21 &
89.4 ± 0.15 &
74.1 ± 0.11 &
---
66.86956697284273 16.370773622340355


<hr style="border: 5px dashed red">

# Domain Classifier

In [7]:
X = DATA["X"]
y = DATA["y"]
category = DATA["category"]

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in enumerate(rskf.split(X, y)):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", category.shape)

domain_model, domain_scores, domain_pca = experiment(foldids, X, 
                                         category, LogisticRegression(max_iter=10000, class_weight='balanced'))

shapes X (186477, 768) y (186477,)
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Score                | Average    | Kat_1      | Kat_2      | Kat_3      | Kat_4      | Kat_5      | Kat_6      |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Accuracy             | 71.3 ± 0.2 | -          | -          | -          | -          | -          | -          |
| Balanced Accuracy    | 80.0 ± 0.2 | -          | -          | -          | -          | -          | -          |
| F1                   | 71.2 ± 0.2 | 94.7 ± 0.2 | 77.2 ± 0.4 | 64.0 ± 0.2 | 64.0 ± 0.2 | 76.4 ± 0.3 | 70.4 ± 0.2 |
| G-mean               | 78.9 ± 0.2 | -          | -          | -          | -          | -          | -          |
| Precision            | 72.3 ± 0.2 | 92.2 ± 0.3 | 67.5 ± 0.5 | 51.1 ± 0.3 | 65.8 ± 0.3 | 72.8 ± 0.5 | 78.4 ± 0.4 |
| Recall               | 71.3 ± 0.2 |

<hr style="border: 5px dashed red">

# ParamMultitaskShare

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class SparseNet(nn.Module):

    def __init__(self, K:int=32, M:int=128, R_lr:float=0.1, lmda:float=5e-3, device=None):
        super(SparseNet, self).__init__()
        self.K = K
        self.M = M
        self.R_lr = R_lr
        self.lmda = lmda
        # synaptic weights
        self.device = torch.device("cpu") if device is None else device
        self.U = nn.Linear(self.K, self.M, bias=False).to(self.device)
        # responses
        self.R = None
        self.normalize_weights()

    def ista_(self, img_batch):
        # create R
        self.R = torch.zeros((img_batch.shape[0], self.K), requires_grad=True, device=self.device)
        converged = False
        # update R
        optim = torch.optim.SGD([{'params': self.R, "lr": self.R_lr}])
        # train
        while not converged:
            old_R = self.R.clone().detach()
            # pred
            pred = self.U(self.R)
            # loss
            loss = ((img_batch - pred) ** 2).sum()
            loss.backward()
            # update R in place
            optim.step()
            # zero grad
            self.zero_grad()
            # prox
            self.R.data = SparseNet.soft_thresholding_(self.R, self.lmda)
            # convergence
            converged = torch.norm(self.R - old_R) / torch.norm(old_R) < 0.01

    @staticmethod
    def soft_thresholding_(x, alpha):
        with torch.no_grad():
            rtn = F.relu(x - alpha) - F.relu(-x - alpha)
        return rtn.data

    def zero_grad(self):
        self.R.grad.zero_()
        self.U.zero_grad()

    def normalize_weights(self):
        with torch.no_grad():
            self.U.weight.data = F.normalize(self.U.weight.data, dim=0)

    def forward(self, img_batch):
        # first fit
        self.ista_(img_batch)
        # now predict again
        pred = self.U(self.R)
        return pred, self.R


from math import log
import numpy as np
from scipy.special import logsumexp
from scipy.linalg import sqrtm, inv, norm
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression, Lasso
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, explained_variance_score
from sklearn.decomposition import PCA

class ParamMultitaskShare(object):
    def __init__(self):
        self.net = SparseNet()
        self.base_learner = LogisticRegression
        self.codes = []

    def fit(self, X, y, task_id):
        single_task_model = self.base_learner(max_iter=10000, class_weight='balanced')
        single_task_model.fit(X,y)
        theta_t = torch.tensor(single_task_model.coef_)
        optim = torch.optim.SGD([{'params': self.net.U.weight, "lr": 1e-2}])
        
        running_loss = 0
        code = None
        for c in range(50):
            pred, code = self.net.forward(theta_t)
            loss = ((theta_t - pred) ** 2).sum()
            running_loss += loss.item()
            loss.backward()
            optim.step()
            self.net.zero_grad()
            self.net.normalize_weights()
            #print('Loss', running_loss / (c+1))
        
        self.codes.append(code.clone().detach().numpy()[0])
#        tpred = self.predict(X,task_id)
#         print()
#         print("PMS-BAC:", balanced_accuracy_score(y,tpred))
#         print("T-BAC:", balanced_accuracy_score(y,single_task_model.predict(X)))

    def predict(self, X, task_id):
        code = self.codes[task_id]
        theta_t = self.net.U(torch.tensor(code)).detach().numpy()
        sign = 1. / (1.0 + np.exp(-X.dot(theta_t)))
        pred = [1 if p > 0.5 else 0 for p in sign]
        return np.array(pred)


# Param Share Performance

In [9]:
T = np.max(DATA["category"])+1
d = 128

bacs = [[] for b in range(T)]


for _foldid in range(10):
    print("fold",_foldid)
    ella = ParamMultitaskShare()

    ### Train
    for task in range(T):
        #print("TASK", task)
        X = DATA["X"][DATA["category"] == task]
        y = DATA["y"][DATA["category"] == task]
        foldids = DATA["folds"][task]
       
        train = foldids[_foldid][1]
        test = foldids[_foldid][2]
        
        pca = PCA(n_components=d)
        pca.fit_transform(X[train])
        X=pca.transform(X[train])


        ella.fit(X, y[train], task)
        
    ##### Test
    for task in range(T):
        X = DATA["X"][DATA["category"] == task]
        y = DATA["y"][DATA["category"] == task]
        foldids = DATA["folds"][task]
        
        train = foldids[_foldid][1]
        test = foldids[_foldid][2]    
        
        domain_pred=domain_model.predict(domain_pca.transform(X[test]))
        pred_task_id = np.argmax(np.unique(domain_pred, return_counts=True)[1])
        
        pca = PCA(n_components=d)
        pca.fit_transform(X[train])
        
        X=pca.transform(X[test])

        #print(np.unique(domain_pred, return_counts=True))
        #print("task",task, pred_task_id)

        pred=ella.predict(X, pred_task_id)
        bac = balanced_accuracy_score(y[test], pred)
        print(task,bac)
        bacs[task].append(bac)



fold 0
0 0.7794857388035306
1 0.8677195730363152
2 0.77578837383621
3 0.7648214916142482
4 0.9863031020955114
5 0.7883114984367685
fold 1
0 0.7713654931046235
1 0.8522885485555168
2 0.7911154274263112
3 0.7765563201381726
4 0.9869026880063785
5 0.7903763432628574
fold 2
0 0.7750204315135665
1 0.8394825385775613
2 0.7787028647719616
3 0.7714799258928865
4 0.985105087844648
5 0.788729402347113
fold 3
0 0.7561081691516474
1 0.8481407355841746
2 0.7895485747069316
3 0.7601865119687106
4 0.9853186782821945
5 0.7905926588854593
fold 4
0 0.7614518633540373
1 0.8576981088293305
2 0.7598928560311808
3 0.7621657677108448
4 0.9847894525399199
5 0.7912634234016523
fold 5
0 0.7830482091351656
1 0.861236802413273
2 0.7936239068645572
3 0.7713701178373675
4 0.985969820636784
5 0.7877845738327256
fold 6
0 0.7538727933965348
1 0.8612658080983873
2 0.8040812463562554
3 0.7647859762891788
4 0.9841629901341229
5 0.7854533824706611
fold 7
0 0.7692009909401214
1 0.8606566887109874
2 0.7787265590752716
3 0.7

In [21]:
for t in range(T):
    print("%3.1f ± %3.1f &" % (np.mean(bacs[t])*100, 100*np.std(bacs[t])) )

print("---")
print(np.mean(np.array(bacs).flatten())*100, np.std(np.array(bacs).flatten())*100)

77.0 ± 1.0 &
85.5 ± 1.1 &
78.4 ± 1.1 &
76.9 ± 0.6 &
98.5 ± 0.2 &
78.9 ± 0.2 &
---
82.51776383072779 7.756120196367812
